In [14]:
import os 
from dotenv import load_dotenv
import json

from pathlib import Path

from mistralai import Mistral

load_dotenv('../.env.local')

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key)

In [15]:
from bill_parser_engine.core.reference_resolver import BillSplitter, ReferenceDetector, TargetArticleIdentifier, ReferenceClassifier, TextRetriever
from bill_parser_engine.core.reference_resolver.models import (
    Reference, ResolvedReference, FlattenedText, 
    BillChunk, TargetArticle, TargetOperationType, ProcessedChunkResult,
    ReferenceSource
)

In [16]:
bill_path = '../data/small_legislative_bill_example.md'

In [17]:
bill_text = Path(bill_path).read_text(encoding='utf-8')
bill_text

"N° 41\n\nSÉNAT\n\nSESSION ORDINAIRE DE 2024-2025\n\n27 janvier 2025\n\nPROPOSITION DE LOI\n\nvisant à lever les contraintes\nà l'exercice du métier d'agriculteur\n\n(procédure accélérée)\n\nLe Sénat a adopté, en première lecture,\naprès engagement de la procédure accélérée,\nla proposition de loi dont la teneur suit :\n\nVoir les numéros :\nSénat : 108 rect., 185 et 186 (2024-2025).\n\n---\n\n# TITRE Iᴱᴿ\n\nMETTRE FIN AUX SURTRANSPOSITIONS ET SURRÉGLEMENTATIONS FRANÇAISES EN MATIÈRE DE PRODUITS PHYTOSANITAIRES\n\n## Article 1ᵉʳ\n\nLe code rural et de la pêche maritime est ainsi modifié :\n\n1° (Supprimé)\n\n2° L'article L. 254-1 est ainsi modifié :\n\n    a) (nouveau) Au 3° du II, les mots : « prévu aux articles L. 254-6-2 et L. 254-6-3 » sont remplacés par les mots : « à l'utilisation des produits phytopharmaceutiques » ;\n\n    b) Le VI est ainsi modifié :\n\n    \t- à la fin de la première phrase, les mots : « incompatible avec celui des activités mentionnées aux 1° ou 2° du II ou 

In [18]:
splitter = BillSplitter()

In [19]:
res = splitter.split(bill_text)
bill_chunk = res[3]
bill_chunk

BillChunk(text="3° bis (nouveau) L'article L. 254-1-1 est ainsi modifié :\n\n    a) Le I est ainsi modifié :\n\n    \t- à la fin du 1°, les mots : « mentionnée aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 du 21 octobre 2009 » ;\n\n    \t- au 2°, les mots : « mentionnée aux 1° ou 2° du II ou au IV de l'article L. 254-1 » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » et, à la fin, les mots : « de ce II » sont remplacés par les mots : « du II de l'article L. 254-1 » ;\n\n    \t- au 3°, les mots : « mentionnée, d'une part, au 3° du II de l'article L. 254-1 et, d'autre part, aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « , d'une part, mentionnée au 3° du II de l'article L. 254-1 et, d'autre part, de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » ;\n\n    b) L

In [20]:
len(res)

14

In [21]:
bill_chunk.text

"3° bis (nouveau) L'article L. 254-1-1 est ainsi modifié :\n\n    a) Le I est ainsi modifié :\n\n    \t- à la fin du 1°, les mots : « mentionnée aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 du 21 octobre 2009 » ;\n\n    \t- au 2°, les mots : « mentionnée aux 1° ou 2° du II ou au IV de l'article L. 254-1 » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » et, à la fin, les mots : « de ce II » sont remplacés par les mots : « du II de l'article L. 254-1 » ;\n\n    \t- au 3°, les mots : « mentionnée, d'une part, au 3° du II de l'article L. 254-1 et, d'autre part, aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « , d'une part, mentionnée au 3° du II de l'article L. 254-1 et, d'autre part, de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » ;\n\n    b) Le II est ainsi 

In [22]:
target_identifier = TargetArticleIdentifier(client)

2025-06-01 19:32:27,668 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/agents "HTTP/1.1 200 OK"


In [23]:
chunk_with_target = target_identifier.identify(bill_chunk)

2025-06-01 19:32:30,108 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/conversations "HTTP/1.1 200 OK"


In [24]:
chunk_with_target

BillChunk(text="3° bis (nouveau) L'article L. 254-1-1 est ainsi modifié :\n\n    a) Le I est ainsi modifié :\n\n    \t- à la fin du 1°, les mots : « mentionnée aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 du 21 octobre 2009 » ;\n\n    \t- au 2°, les mots : « mentionnée aux 1° ou 2° du II ou au IV de l'article L. 254-1 » sont remplacés par les mots : « de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » et, à la fin, les mots : « de ce II » sont remplacés par les mots : « du II de l'article L. 254-1 » ;\n\n    \t- au 3°, les mots : « mentionnée, d'une part, au 3° du II de l'article L. 254-1 et, d'autre part, aux 1° ou 2° du même II ou au IV du même article » sont remplacés par les mots : « , d'une part, mentionnée au 3° du II de l'article L. 254-1 et, d'autre part, de producteur au sens du 11 de l'article 3 du règlement (CE) n° 1107/2009 » ;\n\n    b) L

In [25]:
reference_detector = ReferenceDetector(client=client)

2025-06-01 19:32:30,542 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/agents "HTTP/1.1 200 OK"


In [26]:
references = reference_detector.detect_from_chunk(chunk_with_target)

2025-06-01 19:32:37,618 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/conversations "HTTP/1.1 200 OK"


In [27]:
references

[Reference(text="11 de l'article 3 du règlement (CE) n° 1107/2009 du 21 octobre 2009", start_pos=146, end_pos=202, object='de producteur', reference_type=None, source=None, components={}),
 Reference(text="11 de l'article 3 du règlement (CE) n° 1107/2009", start_pos=289, end_pos=345, object='de producteur', reference_type=None, source=None, components={}),
 Reference(text="II de l'article L. 254-1", start_pos=367, end_pos=385, object='de ce II', reference_type=None, source=None, components={}),
 Reference(text="11 de l'article 3 du règlement (CE) n° 1107/2009", start_pos=456, end_pos=512, object='de producteur', reference_type=None, source=None, components={}),
 Reference(text="3° du II de l'article L. 254-1", start_pos=563, end_pos=587, object='mentionnée', reference_type=None, source=None, components={}),
 Reference(text="11 de l'article 3 du règlement (CE) n° 1107/2009", start_pos=610, end_pos=666, object='de producteur', reference_type=None, source=None, components={}),
 Reference(

In [28]:
classifier = ReferenceClassifier(client)

2025-06-01 19:32:40,372 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/agents "HTTP/1.1 200 OK"


In [38]:
classified_references = classifier.classify_batch(references[2:3], chunk_with_target)

2025-06-01 19:33:07,688 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/conversations "HTTP/1.1 200 OK"


In [39]:
classified_references

[Reference(text="II de l'article L. 254-1", start_pos=367, end_pos=385, object='de ce II', reference_type=<ReferenceType.EXPLICIT_SECTION: 'explicit_section'>, source=<ReferenceSource.FRENCH_CODE: 'french_code'>, components={'code': 'code rural et de la pêche maritime', 'article': 'L. 254-1', 'section': 'II'})]

In [40]:
# Post-processing: Ensure 'code' is present for French code references
for ref in classified_references:
    if (
        getattr(ref, 'source', None) == TargetArticle.__annotations__.get('source', None) or getattr(ref, 'source', None) == 'french_code' or getattr(ref, 'source', None) == 'FRENCH_CODE'
    ):
        # Accept both enum and string for robustness
        if 'code' not in getattr(ref, 'components', {}):
            code_val = getattr(chunk_with_target.target_article, 'code', None)
            if code_val:
                ref.components['code'] = code_val

In [41]:
classified_references

[Reference(text="II de l'article L. 254-1", start_pos=367, end_pos=385, object='de ce II', reference_type=<ReferenceType.EXPLICIT_SECTION: 'explicit_section'>, source=<ReferenceSource.FRENCH_CODE: 'french_code'>, components={'code': 'code rural et de la pêche maritime', 'article': 'L. 254-1', 'section': 'II'})]

In [42]:
retriever = TextRetriever()

In [43]:
ref = classified_references[0]
ref

Reference(text="II de l'article L. 254-1", start_pos=367, end_pos=385, object='de ce II', reference_type=<ReferenceType.EXPLICIT_SECTION: 'explicit_section'>, source=<ReferenceSource.FRENCH_CODE: 'french_code'>, components={'code': 'code rural et de la pêche maritime', 'article': 'L. 254-1', 'section': 'II'})

In [ ]:
retriever.retrieve(ref)

In [44]:
metadata = {
    "api_source": "legifrance",
    "reference_object": ref.object if hasattr(ref, "object") else None
}

In [45]:
retriever._retrieve_french_code(ref, metadata)

2025-06-01 19:33:22,489 - root - INFO - recherche_code params: code_enum=CodeNom.CREDLPM, search_article=L254-1
2025-06-01 19:33:22,670 - pylegifrance.auth - INFO - Legifrance API authentication successful.
2025-06-01 19:33:22,672 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/search
2025-06-01 19:33:23,253 - pylegifrance.client - INFO - API call to 'search' successful.
2025-06-01 19:33:23,254 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1
2025-06-01 19:33:23,254 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle
2025-06-01 19:33:23,461 - pylegifrance.client - INFO - API call to 'consult/getArticle' successful.
2025-06-01 19:33:28,067 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


('"Est subordonné à la détention d\'un agrément l\'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux utilisateurs de ces produits ou aux personnes physiques ou morales agissant pour leur compte, y compris les groupements d\'achats ; 2° L\'application, en qualité de prestataire de services, des produits phytopharmaceutiques, sauf si elle est effectuée dans le cadre de contrats d\'entraide à titre gratuit au sens de l\'article L. 325-1 ou par un exploitant agricole titulaire du certificat mentionné au II de l\'article L. 254-3 sur des exploitations dont la surface agricole utile est inférieure ou égale à la surface définie en application du premier alinéa du V de l\'article L. 732-39 , ou si les produits appliqués sont des produits de biocontrôle définis à l\'article L. 253-6 et ne faisant pas l\'objet d\'une classification mentionnée à l\'article L. 253-4 ou si ces produits sont des produits à faible

In [46]:
ref.components

{'code': 'code rural et de la pêche maritime',
 'article': 'L. 254-1',
 'section': 'II'}

In [49]:
from pylegifrance import recherche_code
from pylegifrance.models.constants import CodeNom

result = recherche_code(code_name=CodeNom.CREDLPM, search='L254-1')


2025-06-01 18:24:24,546 - pylegifrance.auth - INFO - Legifrance API authentication successful.
2025-06-01 18:24:24,548 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/search
2025-06-01 18:24:25,075 - pylegifrance.client - INFO - API call to 'search' successful.
2025-06-01 18:24:25,078 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1
2025-06-01 18:24:25,079 - pylegifrance.client - INFO - POST request to URL: https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticle
2025-06-01 18:24:25,260 - pylegifrance.client - INFO - API call to 'consult/getArticle' successful.


In [50]:
result

[{'executionTime': 0,
  'dereferenced': False,
  'article': {'id': 'LEGIARTI000047453622',
   'idTexte': None,
   'type': 'AUTONOME',
   'texte': "I.-Les produits phytopharmaceutiques mentionnés au présent chapitre sont ceux définis au 1 de l'article 2 du règlement (CE) n° 1107/2009. II.-Est subordonné à la détention d'un agrément l'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux utilisateurs de ces produits ou aux personnes physiques ou morales agissant pour leur compte, y compris les groupements d'achats ; 2° L'application, en qualité de prestataire de services, des produits phytopharmaceutiques, sauf si elle est effectuée dans le cadre de contrats d'entraide à titre gratuit au sens de l'article L. 325-1 ou par un exploitant agricole titulaire du certificat mentionné au II de l'article L. 254-3 sur des exploitations dont la surface agricole utile est inférieure ou égale à la surface définie en a